In [5]:
from huggingface_hub import snapshot_download

In [4]:
snapshot_download(
    repo_id="pietrolesci/fineweb-edu-10BT", repo_type="dataset", local_dir="data", allow_patterns="bpe32000/*"
)

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

bpe32000/000_bpe32000.ds.metadata:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

bpe32000/001_bpe32000.ds.index:   0%|          | 0.00/64.6k [00:00<?, ?B/s]

bpe32000/001_bpe32000.ds.metadata:   0%|          | 0.00/76.0 [00:00<?, ?B/s]

000_bpe32000.ds:   0%|          | 0.00/20.0G [00:00<?, ?B/s]

bpe32000/bpe32000.ds.metadata:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

001_bpe32000.ds:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

000_bpe32000.ds.index:   0%|          | 0.00/77.3M [00:00<?, ?B/s]

'/home/pl487/rdd/data'

In [7]:
from datatrove.utils.dataset import DatatroveFolderDataset

ds = DatatroveFolderDataset(
    folder_path="data/fineweb-edu-10BT/bpe32000/",
    filename_pattern="data/fineweb-edu-10BT/bpe32000/*.ds",
    seq_len=2048,
    shuffle=False,
    seed=42,
    token_size=2,
)

In [23]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM

In [22]:
model = AutoModelForCausalLM.from_pretrained(
    "/home/pl487/rdd/outputs/model_train/pythia_bpe32000_2024-09-19T16-00-04/checkpoints/checkpoint-0"
)
model = model.eval()

In [25]:
batch = next(iter(DataLoader(ds, batch_size=5, shuffle=False)))

In [28]:
with torch.inference_mode():
    input_ids = batch["input_ids"]
    labels = input_ids.clone()
    loss1 = model(input_ids=input_ids, labels=labels).loss

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.45. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/internal/generation_utils#transformers.Cache)


In [34]:
with torch.inference_mode():
    input_ids = batch["input_ids"][:, :-1]
    labels = batch["input_ids"][:, 1:]
    logits = model.forward(input_ids=input_ids).logits
    loss2 = torch.nn.functional.cross_entropy(logits.permute(0, 2, 1), labels)

In [1]:
from datasets import Dataset, load_from_disk

In [50]:
ds = Dataset.from_parquet("/home/pl487/rdd/data/slim-pajama-eval-bpe32000/data.parquet")

In [53]:
ds = ds.shuffle(seed=42)

In [54]:
ds = ds.take(20_000)

In [56]:
ds = ds.select_columns(["uid", "input_ids"])

In [63]:
ds.save_to_disk("/home/pl487/rdd/data/slim-pajama-eval-bpe32000/validation_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/20000 [00:00<?, ? examples/s]

In [11]:
ds = load_from_disk("/home/pl487/rdd/data/slim-pajama-eval-bpe32000/validation_dataset")

In [12]:
df = ds.to_polars()

In [13]:
import polars as pl

In [19]:
df = (
    df.with_columns(pl.col("input_ids").list.slice(0, 2049))
    .with_columns(len=pl.col("input_ids").list.len())
    .sort("len", descending=True)
    .drop("len")
)

In [20]:
ds = Dataset.from_polars(df)

In [21]:
ds.save_to_disk("/home/pl487/rdd/data/slim-pajama-eval-bpe32000/validation_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/20000 [00:00<?, ? examples/s]

In [60]:
pds = ds.to_pandas()

In [62]:
pds["input_ids"].map(len).describe()

count     20000.000000
mean       1112.514700
std        7862.181971
min          24.000000
25%         205.000000
50%         482.000000
75%         983.000000
max      752280.000000
Name: input_ids, dtype: float64

In [35]:
loss2

tensor(10.3929)

In [10]:
from pathlib import Path

import polars as pl
import srsly
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
ds = load_from_disk("/home/pl487/rdd/data/slim-pajama-eval-bpe32000/eval_samples/")
model = AutoModelForCausalLM.from_pretrained(
    "/home/pl487/rdd/outputs/model_train/pythia-9M-bpe32000/checkpoints/checkpoint-50000"
)

In [3]:
batch = torch.tensor(ds[:3]["input_ids"], dtype=torch.long)
batch.shape

torch.Size([3, 2049])

In [4]:
raw_tok_path = Path("/home/pl487/rdd/outputs/tokenizers/bpe32000/")
tok = AutoTokenizer.from_pretrained(raw_tok_path)
prefix_map = {d["prefix"]: d["new_token_id"] for d in srsly.read_jsonl(raw_tok_path / "prefix_map_bpe32000.jsonl")}

In [5]:
# batch_mapping = [
#     prefix_map.get(t, []) + [t] if (i + 1) % 2 == 0 else [t] for i, t in enumerate(batch[:, -2:].flatten().tolist())
# ]
batch_mapping: list[tuple[int, list[int]]] = []
for penultimate_token, last_token in batch[:, -2:].unbind():
    # penultimate token only needs itself (always)
    m_penultimate = penultimate_token.item()

    # for the last token we apply the trick
    m_last = [last_token.item()] + prefix_map.get(last_token.item(), [])

    batch_mapping.append((m_penultimate, m_last))

[' I', ' said', ',', ' "', 'Why', ' would', ' you', ' leave', ' Ann', ' Arbor']

In [6]:
model.eval()
with torch.inference_mode():
    logits = model.forward(input_ids=batch[:, :-1]).logits
    probs = logits.softmax(-1)

last_tokens_probs = probs[:, -2:, :]

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.45. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/internal/generation_utils#transformers.Cache)


In [31]:
tok.batch_decode(batch.numpy()[0, -10:])

[' I', ' said', ',', ' "', 'Why', ' would', ' you', ' leave', ' Ann', ' Arbor']

In [32]:
tok.batch_decode(logits[0, -10:].argmax(-1).numpy())

[' He', "'m", ',', ' "', 'I', ' is', ' you', ' like', ' the', 'ie']

In [ ]:
mask = torch.zeros_like(last_tokens_probs)
mask.shape

In [37]:
mask = torch.zeros_like(last_tokens_probs)
for idx, (m_penultimate, m_last) in enumerate(batch_mapping):
    # penultimate token only gets its position
    mask[idx, -2, m_penultimate] = 1.0

    # last token gets the fix
    mask[idx, -1, m_last] = 1.0

In [ ]:
# check it gets the correct eleements
batch_mapping, (last_tokens_probs * mask).nonzero()

In [ ]:
(last_tokens_probs * mask).sum(-1)

In [ ]:
list(map(len, batch_mapping)), mask.sum(-1)

In [237]:
last_token_ids = t[:, -2:]
last_token_logprobs = logprobs[:, -2:, :]

In [ ]:
tokens = last_token_ids.clone().flatten().cpu().numpy().tolist()
it = last_token_logprobs.reshape(-1, last_token_logprobs.shape[-1])

out = []
for idx, tok in enumerate(tokens):
    # Take other tokens where tok is a prefix + itself
    ids = prefix_map.get(tok, []) + [tok]
    o = it[idx, ids]
    out.append(o)

In [ ]:
tokens

In [ ]:
last_token_logprobs.reshape(-1, last_token_logprobs.shape[-1])

In [ ]:
# mask = torch.zeros_like(last_token_logprobs).reshape(last_token_logprobs.shape[0], -1)
mask = torch.zeros_like(last_token_logprobs).reshape(-1)
mask.shape

In [ ]:
last_token_ids

In [271]:
tokens = last_token_ids.clone().flatten().cpu().numpy().tolist()

pos = []
for tok in tokens:
    # Take other tokens where tok is a prefix + itself
    pos.append(prefix_map.get(tok, []) + [tok])

In [ ]:
pos

In [ ]:
last_token_ids.flatten()

In [ ]:
# batch, token_position_in_seq, token_position_in_vocab
logprobs[0, -2, 6106], logprobs[0, -1, 30681]

In [ ]:
# batch_size, seq_len, added
last_token_ids = t[:, -2:].unsqueeze(-1)
print(last_token_ids.shape)
last_token_ids

In [ ]:
last_token_logprobs = logprobs[:, -2:, :]
print(last_token_logprobs.shape)
last_token_logprobs

In [ ]:
res = last_token_logprobs.take_along_dim(dim=-1, indices=last_token_ids)
print(res.shape)
res.squeeze(-1)

In [ ]:
last_token_logprobs[0, 0]

In [111]:
p_map = {6016: [1, 2]}

In [ ]:
t[..., -2:]

In [ ]:
t[..., -2:]

In [ ]:
torch.tensor([[0], [2, 3]])

In [ ]:
?torch.where

In [72]:
other = torch.nn.functional.cross_entropy(shift_logits.permute(0, 2, 1), shift_labels, reduction="none")

In [ ]:
other

In [2]:
import numpy as np

In [6]:
a = [np.pad([1, 2, 3], (2, 0), constant_values=0), np.pad([1, 2, 3], (2, 0), constant_values=0)]

In [ ]:
np.vstack(a).shape

In [15]:
a = pl.read_parquet("/home/pl487/rdd/outputs/model_eval/2024-09-11T11-06-11/pythia-9M-bpe32000.parquet")
b = pl.read_parquet("/home/pl487/rdd/outputs/model_eval/2024-09-11T11-31-36/pythia-9M-bpe32000.parquet")

In [ ]:
a.join(b, on=["uid", "new_token_id"])

In [ ]:
b

In [3]:
import srsly
from datasets import load_dataset
from transformers import PreTrainedTokenizerFast

from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.processors import ByteLevel as ByteLevelProcessor
from tokenizers.trainers import BpeTrainer

In [3]:
eos_token = "|endoftext|"

# Define the trainer
trainer = BpeTrainer(vocab_size=2000, min_frequency=2, special_tokens=[eos_token], show_progress=True)

# Define the tokenizer
tokenizer = Tokenizer(BPE())

# Set up the tokenizer components
tokenizer.pre_tokenizer = ByteLevel(add_prefix_space=False, trim_offsets=True, use_regex=True)
tokenizer.post_processor = ByteLevelProcessor()
tokenizer.decoder = ByteLevelDecoder()

In [4]:
with open("tokenizers/modified_bpe/data/test_small.txt") as fl:
    lines = fl.readlines()

In [ ]:
num_docs = 10_000
batch_size = 1_000

dataset = load_dataset(
    "HuggingFaceFW/fineweb-edu", "sample-10BT", split="train", streaming=True, cache_dir=".data_cache"
)
dataset = dataset.take(num_docs).select_columns(["text"]).batch(batch_size=batch_size)

In [ ]:
tokenizer.train_from_iterator(iter(x["text"] for x in dataset), trainer, int(num_docs / batch_size))

In [2]:
from pathlib import Path

In [42]:
import json

from tokenizers import Tokenizer

In [44]:
def load_tokenizer_with_vocab_size(path: str | Path, vocab_size: int) -> PreTrainedTokenizerFast:
    path = Path(path)

    # Edit conf to adapt to the new vocab_size
    conf = srsly.read_json(path / "tokenizer.json")
    len_alphabet = len(conf["model"]["vocab"]) - len(conf["model"]["merges"])
    conf["model"]["vocab"] = dict(list(conf["model"]["vocab"].items())[:vocab_size])
    conf["model"]["merges"] = conf["model"]["merges"][: vocab_size - len_alphabet]

    # Instantiate tokenizer using tokenizers library
    backend_tok = Tokenizer.from_str(json.dumps(conf))
    eos_token = srsly.read_yaml(path / "metadata.yaml")["eos_token"]

    # Instantiate PreTrainedTokenizerFast from object
    # NOTE: we do not instantiate from file directly due to compatibility
    # https://github.com/huggingface/tokenizers/issues/1562#issuecomment-2315349846
    tok = PreTrainedTokenizerFast(tokenizer_object=backend_tok)
    tok.padding_side = "left"
    tok.eos_token = eos_token

    return tok

In [45]:
path = Path("outputs/tokenizers/2024-08-28T16-34-11")
vocab_size = 500

tok = load_tokenizer_with_vocab_size(path, vocab_size)

In [ ]:
tok.save_pretrained(path / f"tok-vocab{vocab_size}")

In [34]:
vocab_size = 500

# edit conf to adapt to the new vocab_size
conf = srsly.read_json(path / "tokenizer.json")
# conf["padding"] = "left"
len_alphabet = len(conf["model"]["vocab"]) - len(conf["model"]["merges"])

conf["model"]["vocab"] = dict(list(conf["model"]["vocab"].items())[:vocab_size])
conf["model"]["merges"] = conf["model"]["merges"][: vocab_size - len_alphabet]

In [39]:
t = Tokenizer.from_str(json.dumps(conf))

In [ ]:
conf["model"].keys()

In [ ]:
t.get_vocab_size()

----